# Counterfactuals guided by prototypes on Boston housing dataset

This notebook goes through an example of [prototypical counterfactuals](../methods/CFProto.ipynb) using [k-d trees](https://en.wikipedia.org/wiki/K-d_tree) to build the prototypes. Please check out [this notebook](./cfproto_mnist.ipynb) for a more in-depth application of the method on MNIST using (auto-)encoders and trust scores.

In this example, we will train a simple neural net to predict whether house prices in the Boston area are above the median value or not. We can then find a counterfactual to see which variables need to be changed to increase or decrease a house price above or below the median value.

In [2]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)  # suppress deprecation messages
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.datasets import load_boston
from sklearn.datasets import load_iris
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.tree import DecisionTreeClassifier
from alibi.explainers import CounterFactualProto
from alibi.explainers import CounterFactual
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

## Load and prepare Boston housing dataset

In [8]:
dataset = load_iris()
data = dataset.data
target = dataset.target
feature_names = dataset.feature_names
class_names = list(dataset.target_names)
print(target)
#print(feature_names)
#print(class_names)
#print(dataset)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


Define train and test set

In [30]:
#x_train,y_train = dataset.data[:idx,:], dataset.target[:idx]
#x_test, y_test = dataset.data[idx+1:,:], dataset.target[idx+1:]
x_train, x_test, y_train, y_test = \
            train_test_split(dataset.data, dataset.target, random_state=None)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print(x_train[1])
print(y_train[1])


[6.5 2.8 4.6 1.5]
[0. 1. 0.]


## Train model

In [4]:
np.random.seed(0)
tf.set_random_seed(0)

In [36]:
def nn_model():
    x_in = Input(shape=(4,))
    x = Dense(40, activation='relu')(x_in)
    x = Dense(40, activation='relu')(x)
    x_out = Dense(3, activation='softmax')(x)
    nn = Model(inputs=x_in, outputs=x_out)
    nn.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return nn

In [37]:
nn = nn_model()
nn.summary()
nn.fit(x_train, y_train, batch_size=64, epochs=500, verbose=0)
#nn.save('nn_boston.h5', save_format='h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 4)                 0         
_________________________________________________________________
dense_15 (Dense)             (None, 40)                200       
_________________________________________________________________
dense_16 (Dense)             (None, 40)                1640      
_________________________________________________________________
dense_17 (Dense)             (None, 3)                 123       
Total params: 1,963
Trainable params: 1,963
Non-trainable params: 0
_________________________________________________________________


In [41]:
score = nn.evaluate(x_test, y_test, verbose=0)
print('Test accuracy: ', score[1])


Test accuracy:  0.94736844
[[6.7 3.3 5.7 2.5]]
test:  [[8.7883718e-06 2.3091815e-02 9.7689945e-01]]


## Generate counterfactual guided by the nearest class prototype

Original instance:

In [61]:
X = np.array([[6.7, 3.3, 5.7, 2.5]])
shape = X.shape
print(shape)

(1, 4)


In [57]:
print(x_test[1].reshape((1,) + x_test[1].shape))
print('original: ', nn.predict(np.array([[6.7, 3.3, 5.7, 2.5]])))

print('new: ', nn.predict(np.array([[7.2676477, 3.8370461, 4.979153 , 1.6486421]])))

[[6.7 3.3 5.7 2.5]]
original:  [[8.7883718e-06 2.3091815e-02 9.7689945e-01]]
new:  [[0.00343219 0.9905461  0.00602177]]


Run counterfactual:

In [68]:
# define model
#nn = load_model('nn_boston.h5')
predict_fn = lambda x: nn.predict(x)
# initialize explainer, fit and generate counterfactual
cf = CounterFactual(predict_fn, shape=shape, target_proba=1.0, tol=0.01,
                    target_class=0, max_iter=1000, lam_init=1e-1,
                    max_lam_steps=100, learning_rate_init=0.1)

print(cf)
#cf.fit(x_train)
explanation = cf.explain(X)
print(explanation)

{'X': array([[6.7023945, 3.3273273, 0.6559316, 2.459649 ]], dtype=float32), 'distance': 5.114141464233398, 'lambda': 0.0002575, 'index': 98026, 'class': 0, 'proba': array([[9.9363637e-01, 6.3623656e-03, 1.2695402e-06]], dtype=float32), 'loss': 0.0013573872175689941}, {'X': array([[6.6940503 , 3.3276432 , 0.68503696, 2.4673502 ]], dtype=float32), 'distance': 5.081205368041992, 'lambda': 0.0002575, 'index': 98027, 'class': 0, 'proba': array([[9.9313074e-01, 6.8678618e-03, 1.4403804e-06]], dtype=float32), 'loss': 0.0013555970670845513}, {'X': array([[6.6986737, 3.3223774, 0.7178855, 2.4815505]], dtype=float32), 'distance': 5.024267196655273, 'lambda': 0.0002575, 'index': 98028, 'class': 0, 'proba': array([[9.9243999e-01, 7.5583267e-03, 1.6718627e-06]], dtype=float32), 'loss': 0.0013509026257768893}, {'X': array([[6.7149863, 3.3142836, 0.7457717, 2.5004625]], dtype=float32), 'distance': 4.9839606285095215, 'lambda': 0.0002575, 'index': 98029, 'class': 0, 'proba': array([[9.916027e-01, 8.39

The prediction flipped from 0 (value below the median) to 1 (above the median):

In [65]:
print('Original prediction: {}'.format(explanation['orig_class']))
print('Counterfactual prediction: {}'.format(explanation['cf']['class']))

Original prediction: 2
Counterfactual prediction: 0


Let's take a look at the counterfactual. To make the results more interpretable, we will first undo the pre-processing step and then check where the counterfactual differs from the original instance:

In [66]:
orig = X * sigma + mu
counterfactual = explanation['cf']['X'] * sigma + mu
delta = counterfactual - orig
for i, f in enumerate(feature_names):
    if np.abs(delta[0][i]) > 1e-4:
        print('{}: {}'.format(f, delta[0][i]))

NameError: name 'sigma' is not defined

So in order to increase the house price, the proportion of owner-occupied units built prior to 1940 should decrease by ~11-12%. This is not surprising since the proportion for the observation is very high at 93.6%. Furthermore, the % of the population with "lower status" should decrease by ~5%.

In [22]:
print('% owner-occupied units built prior to 1940: {}'.format(orig[0][5]))
print('% lower status of the population: {}'.format(orig[0][11]))

% owner-occupied units built prior to 1940: 93.6
% lower status of the population: 18.68


Clean up:

In [16]:
os.remove('nn_boston.h5')